# N-Grams and Markov Chains for a Girl

First, read our input text from a file and clean it into a list of words without special characters:

In [99]:
from collections import Counter
from pprint import pprint

def clean(input_text):
    result = input_text

    special_chars = [".", "\n", ";", "?", "!", ":", ",", "(", ")", "[", "]", "\"", "“", "”", "*"]

    for char in special_chars:
        result = result.replace(char, " " if char == "\n" else "")

    return result.lower()


# Clean up the input text
def split_and_dropnulls(input_text):
    words = input_text.split(" ")

    non_empty_words = [word for word in words if word != '']
    return non_empty_words

with open('alice.txt', 'r') as alice_file: 
    alice_text = ' '.join(alice_file.readlines())

alice_words = split_and_dropnulls(clean(alice_text))
pprint(alice_words[:10])

['start',
 'of',
 'the',
 'project',
 'gutenberg',
 'ebook',
 "alice's",
 'adventures',
 'in',
 'wonderland']


Next, let's split the text into pairs of two (later, N) words at a time:

In [100]:
alice_pairs = [(alice_words[i], alice_words[i+1]) for i in range(len(alice_words)-1)]
pprint(alice_pairs[:10])

[('start', 'of'),
 ('of', 'the'),
 ('the', 'project'),
 ('project', 'gutenberg'),
 ('gutenberg', 'ebook'),
 ('ebook', "alice's"),
 ("alice's", 'adventures'),
 ('adventures', 'in'),
 ('in', 'wonderland'),
 ('wonderland', 'cover')]


Now, we can find the frequency of each pair within the set of pairs:

In [101]:
pair_counts = Counter(alice_pairs)

frequencies = pair_counts.most_common(10)
pprint(frequencies)

[(('said', 'the'), 209),
 (('of', 'the'), 132),
 (('said', 'alice'), 115),
 (('in', 'a'), 98),
 (('and', 'the'), 80),
 (('in', 'the'), 79),
 (('it', 'was'), 73),
 (('to', 'the'), 69),
 (('the', 'queen'), 65),
 (('as', 'she'), 61)]


In [102]:
import sys 

def markov_model(sequence: list, n: int = 2):
    """
    Create a Markov model (represented as a dict) from the given input sequence, 
    using N-grams of size {n}
    """
    model = {}
    sequence = list(sequence[:]) + [None]
    for starting_position in range(len(sequence) - n):
        current_ngram = tuple(sequence[starting_position:starting_position + n])
        next_item = sequence[starting_position + n]
        
        if current_ngram not in model: 
            model[current_ngram] = [next_item]
        else:
            model[current_ngram].append(next_item)

    return model

alice_model = markov_model(alice_text, 5)
print(f'Finished training! The final model has size: {sys.getsizeof(alice_model)} bytes')
# pprint(model)

Finished training! The final model has size: 1310808 bytes


Now that we have a Markov model of our text, we can use it to generate more text that "looks like" the source material:

In [103]:
import random

def generate(n, model, start=None, max_length=100):
    if start is None:
        start = random.choice(list(model.keys()))
    
    output = list(start)

    for i in range(max_length):
        start = tuple(output[-n:])
        next_item = random.choice(model[start])

        if next_item is None:
            break
        else:
            output.append(next_item)

    return output


alice_result = generate(5, alice_model, max_length=2000)
for char in alice_result:
    print(char, end="")


red,” she tree inches deepest concert!” and I declare,
            *
 “Come, the shook the arches of being of find of more could all this way of smoking and the Queen’s argument she hear you!” But he went on, Alice.
 
 “It isn’t one; “at least notion is, that Dormouse, which seem to be sending its feet high. “Why, then a really you invented at it make out a piteous tone to looked at it might Footman seemed to herself, and the meet Will these stretched by the conversation the King about me, pleased some otherwise, turned in her handed another.”
 
 Alice waited pale animals with passage: and she hearing at all, beautiful, be hungry to about for all,” Alice.
 
 “No,” said Alice replied ever saw the Cat. “I’ve got up in silent, while the puppy was, and the Mouse’s the Queen.
 
 The Hatter now,” thought it spoke for sneeze, were.
 
 Alice all the Duchess, digging for some to a great made her friends to learn it.” Alice.
 
 Alice, and look and shower of laughed deep voice said Alice turns ou

In [104]:
with open('bible.txt', 'r') as bible_file: 
    bible_text = ' '.join(bible_file.readlines())

# Train a model on it 
bible_model = markov_model(bible_text, n=5)

print(f'Finished training! The final model has size: {sys.getsizeof(bible_model)} bytes')

Finished training! The final model has size: 5242968 bytes


In [105]:
# Generate some more "bible" with it
bible_result = generate(n=5, model=bible_model, max_length=1000)

for char in bible_result:
    print(char, end='')


 reigned to be
 deny him, had him.
 
 20:13 But I pass for her, and whole earth be in goings: the
 top of his handmaid, What prevailed with
 thence after thou enemy,
 former
 the wrought to do enquire any more thee.
 
 119:39 And he whom the Gentiles and the son of the
 greatly image, and is restings of this destruct me: 32:22 This stead of all my vessels
 of the world, even away from a month at Jacob.
 
 13:15 And all
 then her end of Gibeah, and carry thee,
 again: there shall set the sons of silver, and destroy the power, nor the mount Ephraim, were was Elisha said unto them, that him.
 
 27:15 Thomas, one angels of the presentence say unto us, if the house of the
 Father’s sake Martha was against he may become out with a bush burned in his time? why wife, and roll: and thee: for ever.
 
 36:4 And he spoken with him, the Lord: the Amorites and
 Beelzebub cast heard
 shall be affliction saw the weapon, he led that the
 dead: the Reuel, and all Israel shall stand unsearching, nor pol

Now, let's do it to the Grateful Dead. (Do you know the grateful dead?)

In [106]:
with open('gdead.txt', 'r') as dead_file: 
    dead_text = ' '.join(dead_file.readlines())

# Train a model on it 
dead_model = markov_model(dead_text, n=5)

print(f'Finished training! The final model has size: {sys.getsizeof(dead_model)} bytes')

Finished training! The final model has size: 589920 bytes


In [107]:
# Generate some more "dead songs" with it
dead_result = generate(n=5, model=dead_model, max_length=1000)

for char in dead_result:
    print(char, end='')

d red grenadine,
 It's nothin's gettin' on the wall away I didn't have to fly,
 You know, I just out slow
 And this grade.
 Call for I feel a deep or wide, if I had too steel, made to shine, their head a harder to fly,
 You get me up the fly,
 I went his whole round, and keep watching to be the song for very lonely swallows for very long since I've chunder in to chunder with me?
 
 Maybe the same story Rider
 
 So instead I've chuckled like the West Mountain,
 And a goddess of the rainstorm, I ducked myself to move,
 The bastard barely swallows for I feeling a maching I got no heart. You love him don't tell.
 Cost two with a Mexican
 The brakes dynamite to go,
 I turned around, and like the desert sands.
 
 Just out don't be a collector of the axis.
 So if you had to bite,
 That's why if you
 
 Well, you go.
 Ain't got a loving to Kansas City, Kansas City where anything To Try
 
 Well, well your part.
 
 Just as a pistol but try dragging my friend; better but nothing to the hill the so